<a href="https://colab.research.google.com/github/caue22/TCC_Ransonware/blob/main/Algoritmos_Colab/Dataset_balanceado_com_udersampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O dataset.rar "https://raw.githubusercontent.com/Juan-Herrera-Silva/Paper-SENSORS/main/Annex%20B%20-%20Dataset%20initial.rar"
!apt-get install -y unrar
!unrar x dataset.rar

--2025-11-11 14:20:55--  https://raw.githubusercontent.com/Juan-Herrera-Silva/Paper-SENSORS/main/Annex%20B%20-%20Dataset%20initial.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9480716 (9.0M) [application/octet-stream]
Saving to: ‘dataset.rar’

dataset.rar         100%[===================>]   9.04M  --.-KB/s    in 0.06s   

2025-11-11 14:20:55 (156 MB/s) - ‘dataset.rar’ saved [9480716/9480716]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from dataset.rar

Extracting  ed0e3c43-8495-45d6-9fe1-c91f

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.under_sampling import RandomUnderSampler
import joblib

# Carregar o conjunto de dados
file_path = '/content/ed0e3c43-8495-45d6-9fe1-c91f36b9c82b_shuffled.csv'
df = pd.read_csv(file_path, low_memory=False)

# Mapeamento dos rótulos antigos para novos significados
label_mapping = {
    'E': 'ransomware de encriptador',
    'L': 'ransomware de locker',
    'G': 'goodware'
}

df.iloc[:, 1] = df.iloc[:, 1].map(label_mapping)

# Remover linhas com valores NaN
missing_labels = df.iloc[:, 1].isnull().sum()
print(f"Valores NaN após o mapeamento: {missing_labels}")
df.dropna(subset=[df.columns[1]], inplace=True)

# Separar características e rótulos
X = df.iloc[:, 2:]  # Características (todas as colunas, exceto as duas primeiras)
y = df.iloc[:, 1]   # Rótulo (segunda coluna)

# Garantir que X contém apenas dados numéricos
X = X.select_dtypes(include=[float, int])

# Codificar os rótulos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Salvar o LabelEncoder para uso posterior
label_encoder_path = '/content/drive/MyDrive/Modelos/label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_path)
print(f"Label encoder salvo em: {label_encoder_path}")

# Balanceamento com undersampling
undersampler = RandomUnderSampler(random_state=42)
X_balanced, y_balanced = undersampler.fit_resample(X, y_encoded)

# Criar um DataFrame balanceado para salvar
balanced_df = pd.DataFrame(X_balanced, columns=X.columns)
balanced_df.insert(0, 'Label', y_balanced)

# Extrair nome do arquivo original
dataset_name = file_path.split('/')[-1].split('.')[0]

# Salvar o dataset balanceado no Google Drive
output_path = f'/content/drive/MyDrive/Dataset Ransonware/{dataset_name}_undersampled.csv'
balanced_df.to_csv(output_path, index=False)
print(f"Dataset balanceado salvo em: {output_path}")

print(f"Dados balanceados - X: {X_balanced.shape}, y: {y_balanced.shape}")


Valores NaN após o mapeamento: 0
Label encoder salvo em: /content/drive/MyDrive/Modelos/label_encoder.pkl
Dataset balanceado salvo em: /content/drive/MyDrive/Dataset Ransonware/ed0e3c43-8495-45d6-9fe1-c91f36b9c82b_shuffled_undersampled.csv
Dados balanceados - X: (175566, 50), y: (175566,)
